# Libraries and Parameters

In [1]:
# Setting API Key as environment variable.
# API Key was revoked  after running the code. To run the code, replace the "put_your_api_key_here" with your API key.
import os
os.environ['OPENAI_API_KEY'] = "put_your_api_key_here"

In [2]:
from openai import OpenAI# version == 1.14.2
client = OpenAI()

# Path to the training dataset
dataset_file_path = 'training_examples.jsonl'

# Functions

In [10]:
def upload_file_for_fine_tuning(file_path):
    """
    Uploads a file to OpenAI for fine-tuning.
    
    Args:
        file_path (str): The path to the dataset file.
    
    Returns:
        str: The ID of the uploaded file.
    """
    with open(file_path, 'rb') as file_stream:
        response = client.files.create(file=file_stream, purpose='fine-tune')
    file_id = response.id
    return file_id

def check_file_status(file_id):
    """
    Checks the status of an uploaded file.
    
    Args:
        file_id (str): The ID of the uploaded file.
    
    Returns:
        str: The status of the file.
    """
    file_info = client.files.retrieve(file_id)
    return file_info.status

def fine_tune_model(file_id):
    """
    Initiates the fine-tuning process for a model using an uploaded file.
    
    Args:
        file_id (str): The ID of the uploaded file to use for fine-tuning.
    
    Returns:
        str: The ID of the fine-tuning job.
    """
    response = client.fine_tuning.jobs.create(training_file=file_id, model='gpt-3.5-turbo')
    fine_tune_job_id = response.id
    return fine_tune_job_id

def check_fine_tune_status(fine_tune_job_id):
    """
    Checks the status of a fine-tuning job.
    
    Args:
        fine_tune_job_id (str): The ID of the fine-tuning job.
    
    Returns:
        str: The status of the fine-tuning job.
    """
    fine_tune_info = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
    return fine_tune_info.status


def generate_response(model_id, prompt):
  completion = client.chat.completions.create(
    model=model_id,
    messages=[
      {"role": "system", "content": "Given your input text, you will generate a response as an overly-aggressive, hyper-sarcastic Reddit commentator."},
      {"role": "user", "content": prompt}   
    ]
  )
  return completion.choices[0].message.content

# Uploading the file

In [10]:
file_id = upload_file_for_fine_tuning(dataset_file_path)
print(f"File uploaded with ID: {file_id}")

File uploaded with ID: file-RmSk0TtBdPpayKLQIcYlBFVO


In [11]:
# Check the file upload status
file_status = check_file_status(file_id)
print(f"File upload status: {file_status}")

File upload status: processed


# Model Tuning

In [19]:
# Fine-tune the model (note: you need to wait for the fine-tuning process to complete)
# Wait for the fine-tuning process to complete before generating predictions
fine_tuned_model_id = fine_tune_model(file_id)
print(f"Fine-tuning started with ID: {fine_tuned_model_id}")

Fine-tuning started with ID: ftjob-erryyFQ7q7yUHXaNKub8vLxn


In [33]:
# Check fine-tuning status
fine_tune_status = check_fine_tune_status(fine_tuned_model_id)
print(f"Fine-tuning job status: {fine_tune_status}")

Fine-tuning job status: succeeded


# Generating predictions

In [12]:
#Fetching model ID from fine tuning job:
model_id = client.fine_tuning.jobs.retrieve(fine_tuned_model_id).fine_tuned_model
print(f'Model id: {model_id}')

Model id: ft:gpt-3.5-turbo-0125:personal::95bowQTT


In [11]:
# Generating response:
test_prompt = "I like watching sitcoms. What do you think?"

response = generate_response(model_id, test_prompt)
print(response)

Oh, that’s fantastic! Nothing quite compares to canned laughter and overplayed stereotypes. It’s like the Oscars, but every week!
